In [2]:
import torch
import pandas as pd
from unsloth import FastLanguageModel
from datasets import load_from_disk
from tqdm import tqdm
import numpy as np
from PIL import Image

# Define paths
# MODEL_OUTPUT_DIR = "outputs/llama-3.2-11b-surgical-cholecT50---10frames_proc+task"
# BASE_MODEL_NAME = "nvidia/Llama-3.2-11B-Vision-Surgical-CholecT50"

# MODEL_OUTPUT_DIR = "outputs/llama-3.2-11b---10frames_proc+task"
# MODEL_OUTPUT_DIR = "outputs/llama-3.2-11b---10frames_proc+task_strict_instruction"
MODEL_OUTPUT_DIR = "../../outputs/qwen2.5-vl-7b"
# BASE_MODEL_NAME = "nvidia/Llama-3.2-11B-Vision"
BASE_MODEL_NAME = "Qwen/Qwen2.5-VL-7B"
LORA_ADAPTERS_PATH = f"{MODEL_OUTPUT_DIR}/lora_model"
# VALIDATION_DATA_PATH = f"outputs/llama-3.2-11b---10frames_proc+task_strict_instruction/val_ds"
VALIDATION_DATA_PATH = '../../outputs/vllm_datasets/val_ds-qwen_vl'

# Inference configuration
BATCH_SIZE = 4  # Adjust based on your GPU memory
MAX_NEW_TOKENS = 100 # Max tokens to generate for each response

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-10 17:11:01.022699: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-10 17:11:01.055068: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-10 17:11:02.261074: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load the fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = LORA_ADAPTERS_PATH, # The path to our saved LoRA adapters
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Set the model to evaluation mode
model.eval()

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Qwen2_5_VLForConditionalGeneration(
  (model): Qwen2_5_VLModel(
    (visual): Qwen2_5_VisionTransformerPretrainedModel(
      (patch_embed): Qwen2_5_VisionPatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2_5_VLVisionBlock(
          (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
          (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
          (attn): Qwen2_5_VLVisionAttention(
            (qkv): Linear(in_features=1280, out_features=3840, bias=True)
            (proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): Qwen2_5_VLMLP(
            (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
            (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
            (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
            (act_fn): SiLU()

In [ ]:
# Load the validation dataset from disk
val_ds = load_from_disk(VALIDATION_DATA_PATH)

from datasets import Dataset
val_ds = Dataset.from_dict(val_ds[::10])

print("Validation dataset loaded:")
print(val_ds)

Validation dataset loaded:
Dataset({
    features: ['answer', 'cvid', 'instrument', 'action', 'tissue', 'procedure', 'task', 'i', 'second', 'image', 'text'],
    num_rows: 8420
})


In [11]:
# Define the instruction and question templates used during training
# instruction = 'Respond with 1-2 sentences of actionable feedback focused on the most teachable instrument-action-tissue event.'
instruction = "Generate actionabl surgical feedback to a trainee surgeon in the context of robot-assisted urology surgery. Your output must be maximum 1 sentence. The surgical feedback must focus on the most teachable instrument-action-tissue event. The feedback must be concise and to the point. Do not generate any other information or context. Do not generate any additional text or explanation."
question_template = "Procedure: {procedure}\nTask: {task}"

def create_inference_prompt(example):
    """
    Creates a prompt for inference by formatting the user's part of the conversation.
    """
    full_question = f"{instruction}\n{question_template.format(procedure=example['procedure'], task=example['task'])}"
    
    # This structure must match the one used in `formatting_prompts_func` during training
    text_data = [
        {"role": "user", "content": [
            {"type": "image"}, 
            {"type": "text", "text": full_question},
        ]},
    ]
    
    # Use add_generation_prompt=True for inference
    prompt = tokenizer.apply_chat_template(text_data, tokenize=False, add_generation_prompt=True)
    return prompt

print("Example inference prompt:\n", create_inference_prompt(val_ds[0]))

Example inference prompt:
 <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Generate actionabl surgical feedback to a trainee surgeon in the context of robot-assisted urology surgery. Your output must be maximum 1 sentence. The surgical feedback must focus on the most teachable instrument-action-tissue event. The feedback must be concise and to the point. Do not generate any other information or context. Do not generate any additional text or explanation.
Procedure: radical prostatectomy
Task: Endopelvic fascia<|im_end|>
<|im_start|>assistant



In [12]:
def save_results(
    cvids,
    ground_truths,
    predictions,
    procedures,
    tasks,
    idxs,
    seconds
):
    # Create a DataFrame with the results
    results_df = pd.DataFrame({
        'cvid': cvids,
        'ground_truth': ground_truths,
        'prediction': predictions,
        'procedure': procedures,
        'task': tasks,
        'idx': idxs,
        'seconds': seconds
    })

    # Save the DataFrame to a CSV file
    output_csv_path = f"{MODEL_OUTPUT_DIR}/inference_results.csv"
    results_df.to_csv(output_csv_path, index=False)

    print(f"Inference complete. Results saved to {output_csv_path}")

    # Display the first 10 results for a quick check
    print("\nSample Inference Results:")
    pd.set_option('display.max_colwidth', None)
    display(results_df.head(10))

In [14]:
from tqdm import tqdm
predictions = []
ground_truths = []
cvids = []
procedures = []
tasks = []
idxs = []
seconds = []

import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Disable parallelism to avoid warnings

# Process the dataset in batches
for i in tqdm(range(0, len(val_ds), BATCH_SIZE), desc="Running Inference"):
    batch_data = val_ds[i : i + BATCH_SIZE]
    keys = batch_data.keys()
    batch_data = [{key: batch_data[key][j] for key in keys} for j in range(len(batch_data['image']))]
    
    # 1. Prepare batch inputs
    prompts = [create_inference_prompt(ex) for ex in batch_data]
    images = [ex['image'] for ex in batch_data]

    # Convert PIL images to numpy arrays if they are not already
    images_nested = [[np.array(img)] for img in images]

    # 2. Tokenize text and prepare inputs for the model
    inputs = tokenizer(
        text=prompts,
        images=images_nested, # Pass the corrected nested list
        return_tensors="pt",
        padding=True,
    ).to("cuda")

    # 3. Generate text
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS, eos_token_id=tokenizer.eos_token_id, use_cache=False)
    
    # 4. Decode the generated text, skipping special tokens
    decoded_outputs = tokenizer.batch_decode(generated_ids[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)
    
    # 5. Store results
    predictions.extend(decoded_outputs)
    ground_truths.extend([ex['answer'] for ex in batch_data])
    cvids.extend([ex['cvid'] for ex in batch_data])
    procedures.extend([ex['procedure'] for ex in batch_data])
    tasks.extend([ex['task'] for ex in batch_data])
    idxs.extend([ex['i'] for ex in batch_data])
    seconds.extend([ex['second'] for ex in batch_data])
    
    save_results(
        cvids=cvids,
        ground_truths=ground_truths,
        predictions=predictions,
        procedures=procedures,
        tasks=tasks,
        idxs=idxs,
        seconds=seconds
    )

Running Inference:   0%|          | 0/211 [00:00<?, ?it/s]

Inference complete. Results saved to ../../outputs/qwen2.5-vl-7b/inference_results.csv

Sample Inference Results:


,cvid,ground_truth,prediction,procedure,task,idx,seconds
0,c28_s0_1-14-11.avi,"now you can cheat towards the prostate, because it's curving towards the apex, right?",Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,0,0
1,c28_s0_1-14-16.avi,"now that's puboprostatics, be careful, there's veins there",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,1,0
2,c28_s0_1-14-23.avi,another layer you have to get that off,Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,2,0
3,c28_s0_1-14-27.avi,press on the prostate,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,3,0


Running Inference:   0%|          | 1/211 [00:02<08:53,  2.54s/it]

Inference complete. Results saved to ../../outputs/qwen2.5-vl-7b/inference_results.csv

Sample Inference Results:


...

,cvid,ground_truth,prediction,procedure,task,idx,seconds
0,c28_s0_1-14-11.avi,"now you can cheat towards the prostate, because it's curving towards the apex, right?",Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,0,0
1,c28_s0_1-14-16.avi,"now that's puboprostatics, be careful, there's veins there",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,1,0
2,c28_s0_1-14-23.avi,another layer you have to get that off,Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,2,0
3,c28_s0_1-14-27.avi,press on the prostate,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,3,0
4,c28_s0_1-14-46.avi,bring your fourth arm in,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,4,0
5,c28_s0_1-14-50.avi,"ugh, ohh you're spreading into muscle. bring your fourth arm in and pull it over",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,5,0
6,c28_s0_1-15-17.avi,um... sure...,Ensure precise dissection of the endopelvic fascia to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,6,0
7,c28_s0_1-15-36.avi,that's all muscle,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,7,0
8,c28_s0_1-17-10.avi,", you see all these muscle fibers right here? you keep splitting them, I don't know why. right? this is your plane under that bridge of fiber, you know what I mean?",Ensure gentle dissection to avoid injury to the neurovascular bundle.,radical prostatectomy,Endopelvic fascia,8,0
9,c28_s0_1-18-21.avi,good,Ensure precise dissection to avoid injury to surrounding structures while maintaining tension on the endopelvic fascia.,radical prostatectomy,Endopelvic fascia,9,0


Running Inference: 100%|█████████▉| 210/211 [08:01<00:02,  2.01s/it]

Inference complete. Results saved to ../../outputs/qwen2.5-vl-7b/inference_results.csv

Sample Inference Results:


,cvid,ground_truth,prediction,procedure,task,idx,seconds
0,c28_s0_1-14-11.avi,"now you can cheat towards the prostate, because it's curving towards the apex, right?",Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,0,0
1,c28_s0_1-14-16.avi,"now that's puboprostatics, be careful, there's veins there",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,1,0
2,c28_s0_1-14-23.avi,another layer you have to get that off,Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,2,0
3,c28_s0_1-14-27.avi,press on the prostate,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,3,0
4,c28_s0_1-14-46.avi,bring your fourth arm in,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,4,0
5,c28_s0_1-14-50.avi,"ugh, ohh you're spreading into muscle. bring your fourth arm in and pull it over",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,5,0
6,c28_s0_1-15-17.avi,um... sure...,Ensure precise dissection of the endopelvic fascia to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,6,0
7,c28_s0_1-15-36.avi,that's all muscle,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,7,0
8,c28_s0_1-17-10.avi,", you see all these muscle fibers right here? you keep splitting them, I don't know why. right? this is your plane under that bridge of fiber, you know what I mean?",Ensure gentle dissection to avoid injury to the neurovascular bundle.,radical prostatectomy,Endopelvic fascia,8,0
9,c28_s0_1-18-21.avi,good,Ensure precise dissection to avoid injury to surrounding structures while maintaining tension on the endopelvic fascia.,radical prostatectomy,Endopelvic fascia,9,0


Running Inference: 100%|██████████| 211/211 [08:03<00:00,  2.29s/it]


In [15]:
# Create a DataFrame with the results
results_df = pd.DataFrame({
    'cvid': cvids,
    'dialogue': ground_truths,
    'feedback': predictions,
    'procedure': procedures,
    'task': tasks,
    'idx': idxs,
    'seconds': seconds
})

# Save the DataFrame to a CSV file
output_csv_path = f"{MODEL_OUTPUT_DIR}/inference_results.csv"
results_df.to_csv(output_csv_path, index=False)

print(f"Inference complete. Results saved to {output_csv_path}")

# Display the first 10 results for a quick check
print("\nSample Inference Results:")
pd.set_option('display.max_colwidth', None)
display(results_df.head(10))

Inference complete. Results saved to ../../outputs/qwen2.5-vl-7b/inference_results.csv

Sample Inference Results:


,cvid,dialogue,feedback,procedure,task,idx,seconds
0,c28_s0_1-14-11.avi,"now you can cheat towards the prostate, because it's curving towards the apex, right?",Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,0,0
1,c28_s0_1-14-16.avi,"now that's puboprostatics, be careful, there's veins there",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,1,0
2,c28_s0_1-14-23.avi,another layer you have to get that off,Ensure precise dissection of the endopelvic fascia to avoid injury to adjacent structures.,radical prostatectomy,Endopelvic fascia,2,0
3,c28_s0_1-14-27.avi,press on the prostate,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,3,0
4,c28_s0_1-14-46.avi,bring your fourth arm in,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,4,0
5,c28_s0_1-14-50.avi,"ugh, ohh you're spreading into muscle. bring your fourth arm in and pull it over",Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,5,0
6,c28_s0_1-15-17.avi,um... sure...,Ensure precise dissection of the endopelvic fascia to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,6,0
7,c28_s0_1-15-36.avi,that's all muscle,Ensure gentle dissection to avoid injury to surrounding structures.,radical prostatectomy,Endopelvic fascia,7,0
8,c28_s0_1-17-10.avi,", you see all these muscle fibers right here? you keep splitting them, I don't know why. right? this is your plane under that bridge of fiber, you know what I mean?",Ensure gentle dissection to avoid injury to the neurovascular bundle.,radical prostatectomy,Endopelvic fascia,8,0
9,c28_s0_1-18-21.avi,good,Ensure precise dissection to avoid injury to surrounding structures while maintaining tension on the endopelvic fascia.,radical prostatectomy,Endopelvic fascia,9,0
